In [1]:
pip install torchmetrics

Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import scipy.io as sio
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Constants
NumPerElement = 5000  # Limit the number of samples to 100 per folder
sequence_length = 3600  # The length of each sequence (number of time steps)
input_size = 2  # Real and Imaginary components

# Assuming you have 7 classes
num_classes = 7
hidden_size = 128  # Number of hidden units in LSTM

# Folder paths and corresponding labels
folders_and_labels = [
    ('/Users/anuraagthakur/Desktop/RawData/FMCWD1', 0),
    ('/Users/anuraagthakur/Desktop/RawData/FMCWD2', 1),
    ('/Users/anuraagthakur/Desktop/RawData/FMCWD3', 2),
    ('/Users/anuraagthakur/Desktop/RawData/FMCWD4', 3),
    # Add other folders here if needed
]

# Prepare data tensors
im = torch.zeros(NumPerElement * len(folders_and_labels), sequence_length, input_size)  # (samples, time_steps, features)
label = torch.zeros(NumPerElement * len(folders_and_labels))

# Counters for the samples processed
count = -1

# Loop through each folder and load the data
for folder_dir, folder_label in folders_and_labels:
    count_in_folder = 0
    for images in os.listdir(folder_dir):
        if images != '.DS_Store' and count_in_folder < NumPerElement:
            # Load the .mat file
            AA = sio.loadmat(os.path.join(folder_dir, images))
            count_in_folder += 1
            count += 1
            for key, value in AA.items():
                # Store real and imaginary parts into the tensor
                im[count, :, 0] = torch.from_numpy(value.real[0, :])  # Real part
                im[count, :, 1] = torch.from_numpy(value.imag[0, :].copy())  # Imaginary part
            label[count] = folder_label

        if count_in_folder >= NumPerElement:  # Stop processing after 1000 files per folder
            break

# Verify the data shape and labels
print(f"Total samples loaded: {count + 1}")
print(f"Labels for first few samples: {label[:10]}")

# Define the LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # LSTM expects input of shape (batch_size, sequence_length, input_size)
        lstm_out, _ = self.lstm(x)
        # Only take the output from the last time step
        last_hidden_state = lstm_out[:, -1, :]
        out = self.fc(last_hidden_state)
        return out

# Define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the device
model = LSTMModel(input_size=input_size, hidden_size=hidden_size, num_classes=num_classes).to(device)

# Convert to dataset and DataLoader
dataset = TensorDataset(im, label)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    model.train()  # Ensure the model is in training mode
    total_loss = 0.0
    total_samples = 0

    all_preds = []
    all_labels = []

    for i, (data, labels) in enumerate(dataloader):
        # Move data and labels to the same device as the model
        data, labels = data.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(data)

        # Compute the loss
        loss = criterion(outputs, labels.long())  # Ensure labels are of type long for classification
        total_loss += loss.item() * data.size(0)  # Accumulate loss (weighted by batch size)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        preds = torch.argmax(outputs, dim=1)

        # Collect all predictions and labels for metrics
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

        total_samples += data.size(0)

        if (i+1) % 10 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}")

    # Convert all_preds and all_labels to numpy arrays
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Calculate Precision, Recall, F1 for each class
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average=None, labels=np.arange(num_classes))
    recall = recall_score(all_labels, all_preds, average=None, labels=np.arange(num_classes))
    f1 = f1_score(all_labels, all_preds, average=None, labels=np.arange(num_classes))

    # Print the detailed metrics for each class
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {total_loss / total_samples:.4f}, Accuracy: {accuracy:.4f}")
    for i in range(num_classes):
        print(f"Class {i}: Precision = {precision[i]:.4f}, Recall = {recall[i]:.4f}, F1 Score = {f1[i]:.4f}")


Total samples loaded: 20000
Labels for first few samples: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Epoch [1/5], Step [10/625], Loss: 1.8624
Epoch [1/5], Step [20/625], Loss: 1.4224
Epoch [1/5], Step [30/625], Loss: 1.3969
Epoch [1/5], Step [40/625], Loss: 1.3771
Epoch [1/5], Step [50/625], Loss: 1.3786
Epoch [1/5], Step [60/625], Loss: 1.3820
Epoch [1/5], Step [70/625], Loss: 1.3977
Epoch [1/5], Step [80/625], Loss: 1.4313
Epoch [1/5], Step [90/625], Loss: 1.3699
Epoch [1/5], Step [100/625], Loss: 1.3918
Epoch [1/5], Step [110/625], Loss: 1.4000
Epoch [1/5], Step [120/625], Loss: 1.3907
Epoch [1/5], Step [130/625], Loss: 1.4315
Epoch [1/5], Step [140/625], Loss: 1.3962
Epoch [1/5], Step [150/625], Loss: 1.3905
Epoch [1/5], Step [160/625], Loss: 1.3971
Epoch [1/5], Step [170/625], Loss: 1.3841
Epoch [1/5], Step [180/625], Loss: 1.3780
Epoch [1/5], Step [190/625], Loss: 1.3931
Epoch [1/5], Step [200/625], Loss: 1.4128
Epoch [1/5], Step [210/625], Loss: 1.3808
Epoch [1/5], Step [2

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Epoch [1/5], Average Loss: 1.4031, Accuracy: 0.2650
Class 0: Precision = 0.2718, Recall = 0.2960, F1 Score = 0.2834
Class 1: Precision = 0.2576, Recall = 0.2726, F1 Score = 0.2649
Class 2: Precision = 0.2635, Recall = 0.2290, F1 Score = 0.2450
Class 3: Precision = 0.2693, Recall = 0.2622, F1 Score = 0.2657
Class 4: Precision = 0.0000, Recall = 0.0000, F1 Score = 0.0000
Class 5: Precision = 0.0000, Recall = 0.0000, F1 Score = 0.0000
Class 6: Precision = 0.0000, Recall = 0.0000, F1 Score = 0.0000
Epoch [2/5], Step [10/625], Loss: 1.3845
Epoch [2/5], Step [20/625], Loss: 1.3797
Epoch [2/5], Step [30/625], Loss: 1.3841
Epoch [2/5], Step [40/625], Loss: 1.3852
Epoch [2/5], Step [50/625], Loss: 1.3759
Epoch [2/5], Step [60/625], Loss: 1.4825
Epoch [2/5], Step [70/625], Loss: 1.3873
Epoch [2/5], Step [80/625], Loss: 1.3801
Epoch [2/5], Step [90/625], Loss: 1.3883
Epoch [2/5], Step [100/625], Loss: 1.4287
Epoch [2/5], Step [110/625], Loss: 1.3713
Epoch [2/5], Step [120/625], Loss: 1.3703
Epoch

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Epoch [2/5], Average Loss: 1.3520, Accuracy: 0.3257
Class 0: Precision = 0.3681, Recall = 0.4486, F1 Score = 0.4044
Class 1: Precision = 0.2762, Recall = 0.2138, F1 Score = 0.2410
Class 2: Precision = 0.2787, Recall = 0.1966, F1 Score = 0.2306
Class 3: Precision = 0.3411, Recall = 0.4440, F1 Score = 0.3858
Class 4: Precision = 0.0000, Recall = 0.0000, F1 Score = 0.0000
Class 5: Precision = 0.0000, Recall = 0.0000, F1 Score = 0.0000
Class 6: Precision = 0.0000, Recall = 0.0000, F1 Score = 0.0000
Epoch [3/5], Step [10/625], Loss: 1.3993
Epoch [3/5], Step [20/625], Loss: 1.3849
Epoch [3/5], Step [30/625], Loss: 1.3604
Epoch [3/5], Step [40/625], Loss: 1.3229
Epoch [3/5], Step [50/625], Loss: 1.3271
Epoch [3/5], Step [60/625], Loss: 1.2253
Epoch [3/5], Step [70/625], Loss: 1.1174
Epoch [3/5], Step [80/625], Loss: 1.1773
Epoch [3/5], Step [90/625], Loss: 1.8519
Epoch [3/5], Step [100/625], Loss: 1.5562
Epoch [3/5], Step [110/625], Loss: 1.4427
Epoch [3/5], Step [120/625], Loss: 1.4466
Epoch

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Epoch [3/5], Average Loss: 1.2029, Accuracy: 0.4314
Class 0: Precision = 0.5295, Recall = 0.6708, F1 Score = 0.5918
Class 1: Precision = 0.3617, Recall = 0.2390, F1 Score = 0.2878
Class 2: Precision = 0.3472, Recall = 0.1952, F1 Score = 0.2499
Class 3: Precision = 0.4109, Recall = 0.6206, F1 Score = 0.4945
Class 4: Precision = 0.0000, Recall = 0.0000, F1 Score = 0.0000
Class 5: Precision = 0.0000, Recall = 0.0000, F1 Score = 0.0000
Class 6: Precision = 0.0000, Recall = 0.0000, F1 Score = 0.0000
Epoch [4/5], Step [10/625], Loss: 0.7601
Epoch [4/5], Step [20/625], Loss: 0.7362
Epoch [4/5], Step [30/625], Loss: 0.9088
Epoch [4/5], Step [40/625], Loss: 0.7651
Epoch [4/5], Step [50/625], Loss: 0.7135
Epoch [4/5], Step [60/625], Loss: 1.5295
Epoch [4/5], Step [70/625], Loss: 1.0029
Epoch [4/5], Step [80/625], Loss: 1.0986
Epoch [4/5], Step [90/625], Loss: 0.8735
Epoch [4/5], Step [100/625], Loss: 0.7360
Epoch [4/5], Step [110/625], Loss: 0.7729
Epoch [4/5], Step [120/625], Loss: 0.7844
Epoch

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Epoch [4/5], Average Loss: 1.1166, Accuracy: 0.4774
Class 0: Precision = 0.6215, Recall = 0.6706, F1 Score = 0.6451
Class 1: Precision = 0.4239, Recall = 0.3340, F1 Score = 0.3736
Class 2: Precision = 0.3434, Recall = 0.2448, F1 Score = 0.2858
Class 3: Precision = 0.4650, Recall = 0.6604, F1 Score = 0.5457
Class 4: Precision = 0.0000, Recall = 0.0000, F1 Score = 0.0000
Class 5: Precision = 0.0000, Recall = 0.0000, F1 Score = 0.0000
Class 6: Precision = 0.0000, Recall = 0.0000, F1 Score = 0.0000
Epoch [5/5], Step [10/625], Loss: 1.2175
Epoch [5/5], Step [20/625], Loss: 1.1890
Epoch [5/5], Step [30/625], Loss: 1.0998
Epoch [5/5], Step [40/625], Loss: 0.9493
Epoch [5/5], Step [50/625], Loss: 0.9022
Epoch [5/5], Step [60/625], Loss: 1.0677
Epoch [5/5], Step [70/625], Loss: 0.9503
Epoch [5/5], Step [80/625], Loss: 0.8342
Epoch [5/5], Step [90/625], Loss: 0.8470
Epoch [5/5], Step [100/625], Loss: 1.0873
Epoch [5/5], Step [110/625], Loss: 1.4140
Epoch [5/5], Step [120/625], Loss: 1.2951
Epoch

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import scipy.io as sio
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Constants
NumPerElement = 1000  # Limit the number of samples to 100 per folder
sequence_length = 3600  # The length of each sequence (number of time steps)
input_size = 2  # Real and Imaginary components

# Assuming you have 7 classes
num_classes = 7
hidden_size = 128  # Number of hidden units in LSTM

# Folder paths and corresponding labels
folders_and_labels = [
    ('/Users/anuraagthakur/Desktop/RawData/FSK2', 0),
    ('/Users/anuraagthakur/Desktop/RawData/FSK4', 1),
    ('/Users/anuraagthakur/Desktop/RawData/FSK8', 2),
    ('/Users/anuraagthakur/Desktop/RawData/FSK16', 3),
    # Add other folders here if needed
]

# Prepare data tensors
im = torch.zeros(NumPerElement * len(folders_and_labels), sequence_length, input_size)  # (samples, time_steps, features)
label = torch.zeros(NumPerElement * len(folders_and_labels))

# Counters for the samples processed
count = -1

# Loop through each folder and load the data
for folder_dir, folder_label in folders_and_labels:
    count_in_folder = 0
    for images in os.listdir(folder_dir):
        if images != '.DS_Store' and count_in_folder < NumPerElement:
            # Load the .mat file
            AA = sio.loadmat(os.path.join(folder_dir, images))
            count_in_folder += 1
            count += 1
            for key, value in AA.items():
                # Store real and imaginary parts into the tensor
                im[count, :, 0] = torch.from_numpy(value.real[0, :])  # Real part
                im[count, :, 1] = torch.from_numpy(value.imag[0, :].copy())  # Imaginary part
            label[count] = folder_label

        if count_in_folder >= NumPerElement:  # Stop processing after 1000 files per folder
            break

# Verify the data shape and labels
print(f"Total samples loaded: {count + 1}")
print(f"Labels for first few samples: {label[:10]}")

# Define the LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # LSTM expects input of shape (batch_size, sequence_length, input_size)
        lstm_out, _ = self.lstm(x)
        # Only take the output from the last time step
        last_hidden_state = lstm_out[:, -1, :]
        out = self.fc(last_hidden_state)
        return out

# Define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the device
model = LSTMModel(input_size=input_size, hidden_size=hidden_size, num_classes=num_classes).to(device)

# Convert to dataset and DataLoader
dataset = TensorDataset(im, label)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    model.train()  # Ensure the model is in training mode
    total_loss = 0.0
    total_samples = 0

    all_preds = []
    all_labels = []

    for i, (data, labels) in enumerate(dataloader):
        # Move data and labels to the same device as the model
        data, labels = data.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(data)

        # Compute the loss
        loss = criterion(outputs, labels.long())  # Ensure labels are of type long for classification
        total_loss += loss.item() * data.size(0)  # Accumulate loss (weighted by batch size)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        preds = torch.argmax(outputs, dim=1)

        # Collect all predictions and labels for metrics
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

        total_samples += data.size(0)

        if (i+1) % 10 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}")

    # Convert all_preds and all_labels to numpy arrays
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Calculate Precision, Recall, F1 for each class, with zero_division set to 0
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    recall = recall_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    f1 = f1_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)

    # Print the detailed metrics for each class
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {total_loss / total_samples:.4f}, Accuracy: {accuracy:.4f}")
    for i in range(num_classes):
        print(f"Class {i}: Precision = {precision[i]:.4f}, Recall = {recall[i]:.4f}, F1 Score = {f1[i]:.4f}")


Total samples loaded: 4000
Labels for first few samples: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Epoch [1/5], Step [10/125], Loss: 1.8470
Epoch [1/5], Step [20/125], Loss: 1.4415
Epoch [1/5], Step [30/125], Loss: 1.4329
Epoch [1/5], Step [40/125], Loss: 1.3950
Epoch [1/5], Step [50/125], Loss: 1.3771
Epoch [1/5], Step [60/125], Loss: 1.4838
Epoch [1/5], Step [70/125], Loss: 1.4013
Epoch [1/5], Step [80/125], Loss: 1.3973
Epoch [1/5], Step [90/125], Loss: 1.3730
Epoch [1/5], Step [100/125], Loss: 1.3880
Epoch [1/5], Step [110/125], Loss: 1.3820
Epoch [1/5], Step [120/125], Loss: 1.3866
Epoch [1/5], Average Loss: 1.4546, Accuracy: 0.2607
Class 0: Precision = 0.2619, Recall = 0.2810, F1 Score = 0.2711
Class 1: Precision = 0.2629, Recall = 0.2540, F1 Score = 0.2584
Class 2: Precision = 0.2557, Recall = 0.2340, F1 Score = 0.2444
Class 3: Precision = 0.2620, Recall = 0.2740, F1 Score = 0.2678
Class 4: Precision = 0.0000, Recall = 0.0000, F1 Score = 0.0000
Class 5: Precision = 0.0000

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import scipy.io as sio
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Constants
NumPerElement = 25  # Limit the number of samples to 100 per folder
sequence_length = 3600  # The length of each sequence (number of time steps)
input_size = 2  # Real and Imaginary components

# Assuming you have 7 classes
num_classes = 7
hidden_size = 128  # Number of hidden units in LSTM

# Folder paths and corresponding labels
folders_and_labels = [
    ('/Users/anuraagthakur/Desktop/RawData/FSK2', 0),
    ('/Users/anuraagthakur/Desktop/RawData/FSK4', 1),
    ('/Users/anuraagthakur/Desktop/RawData/FSK8', 2),
    ('/Users/anuraagthakur/Desktop/RawData/FSK16', 3),
    # Add other folders here if needed
]

# Prepare data tensors
im = torch.zeros(NumPerElement * len(folders_and_labels), sequence_length, input_size)  # (samples, time_steps, features)
label = torch.zeros(NumPerElement * len(folders_and_labels))

# Counters for the samples processed
count = -1

# Loop through each folder and load the data
for folder_dir, folder_label in folders_and_labels:
    count_in_folder = 0
    print(f"Processing folder: {folder_dir}, Label: {folder_label}")  # Debugging print
    for images in os.listdir(folder_dir):
        if images != '.DS_Store' and count_in_folder < NumPerElement:
            # Load the .mat file
            AA = sio.loadmat(os.path.join(folder_dir, images))
            count_in_folder += 1
            count += 1
            for key, value in AA.items():
                # Store real and imaginary parts into the tensor
                im[count, :, 0] = torch.from_numpy(value.real[0, :])  # Real part
                im[count, :, 1] = torch.from_numpy(value.imag[0, :].copy())  # Imaginary part
            label[count] = folder_label

            # Debugging: Print label assignment for the current sample
            if count_in_folder <= 5:  # Print the first few samples for verification
                print(f"Assigned label {folder_label} to sample {count_in_folder} in folder {folder_dir}")

        if count_in_folder >= NumPerElement:  # Stop processing after 5000 files per folder
            break

# Verify the data shape and labels
print(f"Total samples loaded: {count + 1}")
print(f"Labels for first few samples: {label[:10]}")

# Define the LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # LSTM expects input of shape (batch_size, sequence_length, input_size)
        lstm_out, _ = self.lstm(x)
        # Only take the output from the last time step
        last_hidden_state = lstm_out[:, -1, :]
        out = self.fc(last_hidden_state)
        return out

# Define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the device
model = LSTMModel(input_size=input_size, hidden_size=hidden_size, num_classes=num_classes).to(device)

# Convert to dataset and DataLoader
dataset = TensorDataset(im, label)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    model.train()  # Ensure the model is in training mode
    total_loss = 0.0
    total_samples = 0

    all_preds = []
    all_labels = []

    for i, (data, labels) in enumerate(dataloader):
        # Move data and labels to the same device as the model
        data, labels = data.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(data)

        # Compute the loss
        loss = criterion(outputs, labels.long())  # Ensure labels are of type long for classification
        total_loss += loss.item() * data.size(0)  # Accumulate loss (weighted by batch size)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        preds = torch.argmax(outputs, dim=1)

        # Collect all predictions and labels for metrics
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

        total_samples += data.size(0)

        if (i+1) % 10 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}")

    # Convert all_preds and all_labels to numpy arrays
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Calculate Precision, Recall, F1 for each class, with zero_division set to 0
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    recall = recall_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    f1 = f1_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)

    # Print the detailed metrics for each class
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {total_loss / total_samples:.4f}, Accuracy: {accuracy:.4f}")
    for i in range(num_classes):
        print(f"Class {i}: Precision = {precision[i]:.4f}, Recall = {recall[i]:.4f}, F1 Score = {f1[i]:.4f}")


Processing folder: /Users/anuraagthakur/Desktop/RawData/FSK2, Label: 0
Assigned label 0 to sample 1 in folder /Users/anuraagthakur/Desktop/RawData/FSK2
Assigned label 0 to sample 2 in folder /Users/anuraagthakur/Desktop/RawData/FSK2
Assigned label 0 to sample 3 in folder /Users/anuraagthakur/Desktop/RawData/FSK2
Assigned label 0 to sample 4 in folder /Users/anuraagthakur/Desktop/RawData/FSK2
Assigned label 0 to sample 5 in folder /Users/anuraagthakur/Desktop/RawData/FSK2
Processing folder: /Users/anuraagthakur/Desktop/RawData/FSK4, Label: 1
Assigned label 1 to sample 1 in folder /Users/anuraagthakur/Desktop/RawData/FSK4
Assigned label 1 to sample 2 in folder /Users/anuraagthakur/Desktop/RawData/FSK4
Assigned label 1 to sample 3 in folder /Users/anuraagthakur/Desktop/RawData/FSK4
Assigned label 1 to sample 4 in folder /Users/anuraagthakur/Desktop/RawData/FSK4
Assigned label 1 to sample 5 in folder /Users/anuraagthakur/Desktop/RawData/FSK4
Processing folder: /Users/anuraagthakur/Desktop/

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import scipy.io as sio
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Constants
NumPerElement = 1000  # Limit the number of samples to 10 per folder (for testing)
sequence_length = 300  # The length of each sequence (number of time steps)

input_size = 2  # Real and Imaginary components
hidden_size = 128  # Number of hidden units per LSTM layer
num_classes = 7  # Number of output classes
num_layers = 3  # Number of stacked LSTM layers


# Folder paths and corresponding labels
folders_and_labels = [
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/AWGN',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/ImpulseNoise',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/PinkNoise',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/RayleighNoise',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/RicianNoise',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/AWGN',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/ImpulseNoise',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/PinkNoise',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/RayleighNoise',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/RicianNoise',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/AWGN',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/ImpulseNoise',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/PinkNoise',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/RayleighNoise',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/RicianNoise',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/AWGN',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/ImpulseNoise',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/PinkNoise',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/RayleighNoise',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/RicianNoise',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/AWGN',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/ImpulseNoise',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/PinkNoise',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/RayleighNoise',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/RicianNoise',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/AWGN',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/ImpulseNoise',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/PinkNoise',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/RayleighNoise',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/RicianNoise',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/AWGN',6),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/ImpulseNoise',6),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/PinkNoise',6),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/RayleighNoise',6),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/RicianNoise',6),
]

# Prepare data tensors
im = torch.zeros(NumPerElement * len(folders_and_labels), sequence_length, input_size)  # (samples, time_steps, features)
label = torch.zeros(NumPerElement * len(folders_and_labels))

# Counters for the samples processed
count = -1

# Loop through each folder and load the data
for folder_dir, folder_label in folders_and_labels:
    count_in_folder = 0
    for images in os.listdir(folder_dir):
        if images != '.DS_Store' and count_in_folder < NumPerElement:
            # Load the .mat file
            AA = sio.loadmat(os.path.join(folder_dir, images))
            count_in_folder += 1
            count += 1
            for key, value in AA.items():
                if isinstance(value, np.ndarray) and np.iscomplexobj(value):
                    im[count, :, 0] = torch.from_numpy(value.real[0, :])  # Real part
                    im[count, :, 1] = torch.from_numpy(value.imag[0, :].copy())  # Imaginary part
                label[count] = folder_label

        if count_in_folder >= NumPerElement:  # Stop processing after 10 files per folder
            break

# Verify the data shape and labels
print(f"Total samples loaded: {count + 1}")
print(f"Labels for first few samples: {label[:10]}")

# Define the Sequential LSTM Model
model = nn.Sequential(
    nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True),  # Stacked LSTM layers without dropout
    nn.Linear(hidden_size, num_classes)  # Fully connected layer for classification
)

# Define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the device
model = model.to(device)

# Convert to dataset and DataLoader
dataset = TensorDataset(im, label)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    model.train()  # Ensure the model is in training mode
    total_loss = 0.0
    total_samples = 0

    all_preds = []
    all_labels = []

    for i, (data, labels) in enumerate(dataloader):
        # Move data and labels to the same device as the model
        data, labels = data.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        lstm_out, _ = model[0](data)  # Only pass through LSTM layer
        outputs = model[1](lstm_out[:, -1, :])  # Use output from last time step (last hidden state)

        # Compute the loss
        loss = criterion(outputs, labels.long())  # Ensure labels are of type long for classification
        total_loss += loss.item() * data.size(0)  # Accumulate loss (weighted by batch size)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        preds = torch.argmax(outputs, dim=1)

        # Collect all predictions and labels for metrics
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

        total_samples += data.size(0)

        if (i+1) % 10 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}")

    # Convert all_preds and all_labels to numpy arrays
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Calculate Precision, Recall, F1 for each class, with zero_division set to 0
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    recall = recall_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    f1 = f1_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)

    # Print the detailed metrics for each class
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {total_loss / total_samples:.4f}, Accuracy: {accuracy:.4f}")
    for i in range(num_classes):
        print(f"Class {i}: Precision = {precision[i]:.4f}, Recall = {recall[i]:.4f}, F1 Score = {f1[i]:.4f}")


Total samples loaded: 35000
Labels for first few samples: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Epoch [1/5], Step [10/1094], Loss: 1.8670
Epoch [1/5], Step [20/1094], Loss: 1.3991
Epoch [1/5], Step [30/1094], Loss: 1.1393
Epoch [1/5], Step [40/1094], Loss: 0.7881
Epoch [1/5], Step [50/1094], Loss: 0.7944
Epoch [1/5], Step [60/1094], Loss: 0.6789
Epoch [1/5], Step [70/1094], Loss: 1.0009
Epoch [1/5], Step [80/1094], Loss: 0.5479
Epoch [1/5], Step [90/1094], Loss: 0.5807
Epoch [1/5], Step [100/1094], Loss: 0.5564
Epoch [1/5], Step [110/1094], Loss: 0.5584
Epoch [1/5], Step [120/1094], Loss: 0.5950
Epoch [1/5], Step [130/1094], Loss: 0.5901
Epoch [1/5], Step [140/1094], Loss: 0.4778
Epoch [1/5], Step [150/1094], Loss: 0.4623
Epoch [1/5], Step [160/1094], Loss: 0.7225
Epoch [1/5], Step [170/1094], Loss: 0.2783
Epoch [1/5], Step [180/1094], Loss: 0.6150
Epoch [1/5], Step [190/1094], Loss: 0.5647
Epoch [1/5], Step [200/1094], Loss: 0.6302
Epoch [1/5], Step [210/1094], Loss: 0.6970

Epoch [2/5], Step [690/1094], Loss: 0.4831
Epoch [2/5], Step [700/1094], Loss: 0.7619
Epoch [2/5], Step [710/1094], Loss: 0.4106
Epoch [2/5], Step [720/1094], Loss: 0.3815
Epoch [2/5], Step [730/1094], Loss: 0.4959
Epoch [2/5], Step [740/1094], Loss: 0.3357
Epoch [2/5], Step [750/1094], Loss: 0.5090
Epoch [2/5], Step [760/1094], Loss: 0.4047
Epoch [2/5], Step [770/1094], Loss: 0.5701
Epoch [2/5], Step [780/1094], Loss: 0.6265
Epoch [2/5], Step [790/1094], Loss: 0.4955
Epoch [2/5], Step [800/1094], Loss: 0.6111
Epoch [2/5], Step [810/1094], Loss: 0.5579
Epoch [2/5], Step [820/1094], Loss: 0.8465
Epoch [2/5], Step [830/1094], Loss: 0.2990
Epoch [2/5], Step [840/1094], Loss: 0.3088
Epoch [2/5], Step [850/1094], Loss: 0.1843
Epoch [2/5], Step [860/1094], Loss: 0.4186
Epoch [2/5], Step [870/1094], Loss: 0.3510
Epoch [2/5], Step [880/1094], Loss: 0.3838
Epoch [2/5], Step [890/1094], Loss: 0.2429
Epoch [2/5], Step [900/1094], Loss: 0.6500
Epoch [2/5], Step [910/1094], Loss: 0.3150
Epoch [2/5]

Epoch [4/5], Step [190/1094], Loss: 0.2228
Epoch [4/5], Step [200/1094], Loss: 0.3952
Epoch [4/5], Step [210/1094], Loss: 0.3376
Epoch [4/5], Step [220/1094], Loss: 0.1851
Epoch [4/5], Step [230/1094], Loss: 0.3663
Epoch [4/5], Step [240/1094], Loss: 0.4113
Epoch [4/5], Step [250/1094], Loss: 0.3138
Epoch [4/5], Step [260/1094], Loss: 0.3214
Epoch [4/5], Step [270/1094], Loss: 0.2849
Epoch [4/5], Step [280/1094], Loss: 0.1517
Epoch [4/5], Step [290/1094], Loss: 0.6189
Epoch [4/5], Step [300/1094], Loss: 0.3751
Epoch [4/5], Step [310/1094], Loss: 0.2923
Epoch [4/5], Step [320/1094], Loss: 0.6171
Epoch [4/5], Step [330/1094], Loss: 0.4108
Epoch [4/5], Step [340/1094], Loss: 0.1566
Epoch [4/5], Step [350/1094], Loss: 0.2633
Epoch [4/5], Step [360/1094], Loss: 0.5087
Epoch [4/5], Step [370/1094], Loss: 0.6518
Epoch [4/5], Step [380/1094], Loss: 0.4854
Epoch [4/5], Step [390/1094], Loss: 0.2003
Epoch [4/5], Step [400/1094], Loss: 0.3742
Epoch [4/5], Step [410/1094], Loss: 0.4563
Epoch [4/5]

Epoch [5/5], Step [890/1094], Loss: 0.2800
Epoch [5/5], Step [900/1094], Loss: 0.4499
Epoch [5/5], Step [910/1094], Loss: 0.5055
Epoch [5/5], Step [920/1094], Loss: 0.4368
Epoch [5/5], Step [930/1094], Loss: 0.3402
Epoch [5/5], Step [940/1094], Loss: 0.4653
Epoch [5/5], Step [950/1094], Loss: 0.3260
Epoch [5/5], Step [960/1094], Loss: 0.4947
Epoch [5/5], Step [970/1094], Loss: 0.4295
Epoch [5/5], Step [980/1094], Loss: 0.4105
Epoch [5/5], Step [990/1094], Loss: 0.2305
Epoch [5/5], Step [1000/1094], Loss: 0.3259
Epoch [5/5], Step [1010/1094], Loss: 0.5336
Epoch [5/5], Step [1020/1094], Loss: 0.5123
Epoch [5/5], Step [1030/1094], Loss: 0.3820
Epoch [5/5], Step [1040/1094], Loss: 0.3015
Epoch [5/5], Step [1050/1094], Loss: 0.4996
Epoch [5/5], Step [1060/1094], Loss: 0.3119
Epoch [5/5], Step [1070/1094], Loss: 0.5029
Epoch [5/5], Step [1080/1094], Loss: 0.5742
Epoch [5/5], Step [1090/1094], Loss: 0.6207
Epoch [5/5], Average Loss: 0.4097, Accuracy: 0.8517
Class 0: Precision = 0.7930, Recall

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import scipy.io as sio
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Constants
NumPerElement = 1000  # Limit the number of samples to 10 per folder (for testing)
sequence_length = 300  # The length of each sequence (number of time steps)

input_size = 2  # Real and Imaginary components
hidden_size = 128  # Number of hidden units per LSTM layer
num_classes = 7  # Number of output classes
num_layers = 3  # Number of stacked LSTM layers


# Folder paths and corresponding labels
folders_and_labels = [
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/AWGN',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/ImpulseNoise',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/PinkNoise',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/RayleighNoise',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/RicianNoise',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/AWGN',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/ImpulseNoise',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/PinkNoise',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/RayleighNoise',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/RicianNoise',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/AWGN',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/ImpulseNoise',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/PinkNoise',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/RayleighNoise',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/RicianNoise',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/AWGN',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/ImpulseNoise',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/PinkNoise',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/RayleighNoise',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/RicianNoise',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/AWGN',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/ImpulseNoise',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/PinkNoise',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/RayleighNoise',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/RicianNoise',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/AWGN',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/ImpulseNoise',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/PinkNoise',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/RayleighNoise',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/RicianNoise',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/AWGN',6),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/ImpulseNoise',6),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/PinkNoise',6),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/RayleighNoise',6),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/RicianNoise',6),
]

# Prepare data tensors
im = torch.zeros(NumPerElement * len(folders_and_labels), sequence_length, input_size)  # (samples, time_steps, features)
label = torch.zeros(NumPerElement * len(folders_and_labels))

# Counters for the samples processed
count = -1

# Loop through each folder and load the data
for folder_dir, folder_label in folders_and_labels:
    count_in_folder = 0
    for images in os.listdir(folder_dir):
        if images != '.DS_Store' and count_in_folder < NumPerElement:
            # Load the .mat file
            AA = sio.loadmat(os.path.join(folder_dir, images))
            count_in_folder += 1
            count += 1
            for key, value in AA.items():
                if isinstance(value, np.ndarray) and np.iscomplexobj(value):
                    im[count, :, 0] = torch.from_numpy(value.real[0, :])  # Real part
                    im[count, :, 1] = torch.from_numpy(value.imag[0, :].copy())  # Imaginary part
                label[count] = folder_label

        if count_in_folder >= NumPerElement:  # Stop processing after 10 files per folder
            break

# Verify the data shape and labels
print(f"Total samples loaded: {count + 1}")
print(f"Labels for first few samples: {label[:10]}")

# Define the Sequential LSTM Model
model = nn.Sequential(
    nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True),  # Stacked LSTM layers without dropout
    nn.Linear(hidden_size, num_classes)  # Fully connected layer for classification
)

# Define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the device
model = model.to(device)

# Convert to dataset and DataLoader
dataset = TensorDataset(im, label)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    model.train()  # Ensure the model is in training mode
    total_loss = 0.0
    total_samples = 0

    all_preds = []
    all_labels = []

    for i, (data, labels) in enumerate(dataloader):
        # Move data and labels to the same device as the model
        data, labels = data.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        lstm_out, _ = model[0](data)  # Only pass through LSTM layer
        outputs = model[1](lstm_out[:, -1, :])  # Use output from last time step (last hidden state)

        # Compute the loss
        loss = criterion(outputs, labels.long())  # Ensure labels are of type long for classification
        total_loss += loss.item() * data.size(0)  # Accumulate loss (weighted by batch size)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        preds = torch.argmax(outputs, dim=1)

        # Collect all predictions and labels for metrics
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

        total_samples += data.size(0)

        if (i+1) % 10 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}")

    # Convert all_preds and all_labels to numpy arrays
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Calculate Precision, Recall, F1 for each class, with zero_division set to 0
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    recall = recall_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    f1 = f1_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)

    # Print the detailed metrics for each class
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {total_loss / total_samples:.4f}, Accuracy: {accuracy:.4f}")
    for i in range(num_classes):
        print(f"Class {i}: Precision = {precision[i]:.4f}, Recall = {recall[i]:.4f}, F1 Score = {f1[i]:.4f}")


Total samples loaded: 35000
Labels for first few samples: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Epoch [1/10], Step [10/1094], Loss: 1.8991
Epoch [1/10], Step [20/1094], Loss: 1.3588
Epoch [1/10], Step [30/1094], Loss: 1.0488
Epoch [1/10], Step [40/1094], Loss: 0.8190
Epoch [1/10], Step [50/1094], Loss: 0.6895
Epoch [1/10], Step [60/1094], Loss: 0.6875
Epoch [1/10], Step [70/1094], Loss: 0.8302
Epoch [1/10], Step [80/1094], Loss: 0.6546
Epoch [1/10], Step [90/1094], Loss: 0.4984
Epoch [1/10], Step [100/1094], Loss: 0.4419
Epoch [1/10], Step [110/1094], Loss: 0.4563
Epoch [1/10], Step [120/1094], Loss: 0.5440
Epoch [1/10], Step [130/1094], Loss: 0.4542
Epoch [1/10], Step [140/1094], Loss: 0.7560
Epoch [1/10], Step [150/1094], Loss: 0.5661
Epoch [1/10], Step [160/1094], Loss: 0.4915
Epoch [1/10], Step [170/1094], Loss: 0.5513
Epoch [1/10], Step [180/1094], Loss: 0.5827
Epoch [1/10], Step [190/1094], Loss: 0.7946
Epoch [1/10], Step [200/1094], Loss: 0.4629
Epoch [1/10], Step [21

Epoch [2/10], Step [650/1094], Loss: 0.6089
Epoch [2/10], Step [660/1094], Loss: 0.5007
Epoch [2/10], Step [670/1094], Loss: 1.5829
Epoch [2/10], Step [680/1094], Loss: 0.7758
Epoch [2/10], Step [690/1094], Loss: 0.6286
Epoch [2/10], Step [700/1094], Loss: 0.7564
Epoch [2/10], Step [710/1094], Loss: 0.6565
Epoch [2/10], Step [720/1094], Loss: 0.7567
Epoch [2/10], Step [730/1094], Loss: 0.5518
Epoch [2/10], Step [740/1094], Loss: 0.6753
Epoch [2/10], Step [750/1094], Loss: 0.6112
Epoch [2/10], Step [760/1094], Loss: 0.6604
Epoch [2/10], Step [770/1094], Loss: 0.5033
Epoch [2/10], Step [780/1094], Loss: 0.8037
Epoch [2/10], Step [790/1094], Loss: 0.4376
Epoch [2/10], Step [800/1094], Loss: 0.6568
Epoch [2/10], Step [810/1094], Loss: 0.5485
Epoch [2/10], Step [820/1094], Loss: 0.5548
Epoch [2/10], Step [830/1094], Loss: 0.6067
Epoch [2/10], Step [840/1094], Loss: 0.6742
Epoch [2/10], Step [850/1094], Loss: 0.4557
Epoch [2/10], Step [860/1094], Loss: 0.6092
Epoch [2/10], Step [870/1094], L

Epoch [4/10], Step [110/1094], Loss: 0.3965
Epoch [4/10], Step [120/1094], Loss: 0.6074
Epoch [4/10], Step [130/1094], Loss: 0.4563
Epoch [4/10], Step [140/1094], Loss: 0.2877
Epoch [4/10], Step [150/1094], Loss: 0.2364
Epoch [4/10], Step [160/1094], Loss: 0.4800
Epoch [4/10], Step [170/1094], Loss: 0.4053
Epoch [4/10], Step [180/1094], Loss: 0.3927
Epoch [4/10], Step [190/1094], Loss: 0.2620
Epoch [4/10], Step [200/1094], Loss: 0.3501
Epoch [4/10], Step [210/1094], Loss: 0.4240
Epoch [4/10], Step [220/1094], Loss: 0.3138
Epoch [4/10], Step [230/1094], Loss: 0.3414
Epoch [4/10], Step [240/1094], Loss: 0.1195
Epoch [4/10], Step [250/1094], Loss: 0.6797
Epoch [4/10], Step [260/1094], Loss: 0.6759
Epoch [4/10], Step [270/1094], Loss: 0.3606
Epoch [4/10], Step [280/1094], Loss: 0.6565
Epoch [4/10], Step [290/1094], Loss: 0.5052
Epoch [4/10], Step [300/1094], Loss: 0.3337
Epoch [4/10], Step [310/1094], Loss: 0.3782
Epoch [4/10], Step [320/1094], Loss: 0.3714
Epoch [4/10], Step [330/1094], L

Epoch [5/10], Step [770/1094], Loss: 0.4065
Epoch [5/10], Step [780/1094], Loss: 0.5657
Epoch [5/10], Step [790/1094], Loss: 0.3637
Epoch [5/10], Step [800/1094], Loss: 0.4585
Epoch [5/10], Step [810/1094], Loss: 0.5363
Epoch [5/10], Step [820/1094], Loss: 0.8376
Epoch [5/10], Step [830/1094], Loss: 0.4516
Epoch [5/10], Step [840/1094], Loss: 0.6136
Epoch [5/10], Step [850/1094], Loss: 0.5768
Epoch [5/10], Step [860/1094], Loss: 0.5629
Epoch [5/10], Step [870/1094], Loss: 0.2585
Epoch [5/10], Step [880/1094], Loss: 0.3910
Epoch [5/10], Step [890/1094], Loss: 0.3210
Epoch [5/10], Step [900/1094], Loss: 0.3130
Epoch [5/10], Step [910/1094], Loss: 0.2564
Epoch [5/10], Step [920/1094], Loss: 0.5883
Epoch [5/10], Step [930/1094], Loss: 0.1863
Epoch [5/10], Step [940/1094], Loss: 0.7749
Epoch [5/10], Step [950/1094], Loss: 0.9210
Epoch [5/10], Step [960/1094], Loss: 0.6624
Epoch [5/10], Step [970/1094], Loss: 0.3340
Epoch [5/10], Step [980/1094], Loss: 0.2739
Epoch [5/10], Step [990/1094], L

Epoch [7/10], Step [230/1094], Loss: 0.5787
Epoch [7/10], Step [240/1094], Loss: 0.5514
Epoch [7/10], Step [250/1094], Loss: 0.3602
Epoch [7/10], Step [260/1094], Loss: 0.5248
Epoch [7/10], Step [270/1094], Loss: 0.7202
Epoch [7/10], Step [280/1094], Loss: 0.6539
Epoch [7/10], Step [290/1094], Loss: 0.6250
Epoch [7/10], Step [300/1094], Loss: 0.7763
Epoch [7/10], Step [310/1094], Loss: 1.0352
Epoch [7/10], Step [320/1094], Loss: 1.0310
Epoch [7/10], Step [330/1094], Loss: 0.5838
Epoch [7/10], Step [340/1094], Loss: 0.3581
Epoch [7/10], Step [350/1094], Loss: 0.5044
Epoch [7/10], Step [360/1094], Loss: 0.5294
Epoch [7/10], Step [370/1094], Loss: 0.5002
Epoch [7/10], Step [380/1094], Loss: 0.4186
Epoch [7/10], Step [390/1094], Loss: 0.6228
Epoch [7/10], Step [400/1094], Loss: 0.5671
Epoch [7/10], Step [410/1094], Loss: 0.7191
Epoch [7/10], Step [420/1094], Loss: 0.3253
Epoch [7/10], Step [430/1094], Loss: 0.5023
Epoch [7/10], Step [440/1094], Loss: 0.5832
Epoch [7/10], Step [450/1094], L

Epoch [8/10], Step [890/1094], Loss: 0.4312
Epoch [8/10], Step [900/1094], Loss: 0.2721
Epoch [8/10], Step [910/1094], Loss: 0.3672
Epoch [8/10], Step [920/1094], Loss: 0.4203
Epoch [8/10], Step [930/1094], Loss: 0.5314
Epoch [8/10], Step [940/1094], Loss: 0.4168
Epoch [8/10], Step [950/1094], Loss: 0.7801
Epoch [8/10], Step [960/1094], Loss: 0.2023
Epoch [8/10], Step [970/1094], Loss: 0.6936
Epoch [8/10], Step [980/1094], Loss: 0.4022
Epoch [8/10], Step [990/1094], Loss: 0.5211
Epoch [8/10], Step [1000/1094], Loss: 0.5092
Epoch [8/10], Step [1010/1094], Loss: 0.3709
Epoch [8/10], Step [1020/1094], Loss: 0.4139
Epoch [8/10], Step [1030/1094], Loss: 0.4089
Epoch [8/10], Step [1040/1094], Loss: 0.3444
Epoch [8/10], Step [1050/1094], Loss: 0.8351
Epoch [8/10], Step [1060/1094], Loss: 0.5433
Epoch [8/10], Step [1070/1094], Loss: 0.1871
Epoch [8/10], Step [1080/1094], Loss: 0.7076
Epoch [8/10], Step [1090/1094], Loss: 0.3049
Epoch [8/10], Average Loss: 0.4707, Accuracy: 0.8319
Class 0: Prec

Epoch [10/10], Step [340/1094], Loss: 0.3182
Epoch [10/10], Step [350/1094], Loss: 0.3191
Epoch [10/10], Step [360/1094], Loss: 0.5005
Epoch [10/10], Step [370/1094], Loss: 0.6640
Epoch [10/10], Step [380/1094], Loss: 0.4195
Epoch [10/10], Step [390/1094], Loss: 0.7870
Epoch [10/10], Step [400/1094], Loss: 0.5178
Epoch [10/10], Step [410/1094], Loss: 0.4687
Epoch [10/10], Step [420/1094], Loss: 0.5220
Epoch [10/10], Step [430/1094], Loss: 0.4513
Epoch [10/10], Step [440/1094], Loss: 0.4466
Epoch [10/10], Step [450/1094], Loss: 0.4134
Epoch [10/10], Step [460/1094], Loss: 0.5796
Epoch [10/10], Step [470/1094], Loss: 0.4052
Epoch [10/10], Step [480/1094], Loss: 0.3233
Epoch [10/10], Step [490/1094], Loss: 0.4446
Epoch [10/10], Step [500/1094], Loss: 0.4643
Epoch [10/10], Step [510/1094], Loss: 0.4567
Epoch [10/10], Step [520/1094], Loss: 0.5329
Epoch [10/10], Step [530/1094], Loss: 0.3009
Epoch [10/10], Step [540/1094], Loss: 0.3184
Epoch [10/10], Step [550/1094], Loss: 0.3948
Epoch [10/

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import scipy.io as sio
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Constants
NumPerElement = 1000  # Limit the number of samples to 10 per folder (for testing)
sequence_length = 300  # The length of each sequence (number of time steps)

input_size = 2  # Real and Imaginary components
hidden_size = 130  # Number of hidden units per LSTM layer
num_classes = 7  # Number of output classes
num_layers = 6  # Number of stacked LSTM layers


# Folder paths and corresponding labels
folders_and_labels = [
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/AWGN',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/ImpulseNoise',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/PinkNoise',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/RayleighNoise',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/RicianNoise',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/AWGN',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/ImpulseNoise',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/PinkNoise',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/RayleighNoise',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/RicianNoise',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/AWGN',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/ImpulseNoise',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/PinkNoise',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/RayleighNoise',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/RicianNoise',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/AWGN',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/ImpulseNoise',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/PinkNoise',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/RayleighNoise',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/RicianNoise',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/AWGN',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/ImpulseNoise',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/PinkNoise',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/RayleighNoise',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/RicianNoise',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/AWGN',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/ImpulseNoise',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/PinkNoise',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/RayleighNoise',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/RicianNoise',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/AWGN',6),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/ImpulseNoise',6),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/PinkNoise',6),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/RayleighNoise',6),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/RicianNoise',6),
]

# Prepare data tensors
im = torch.zeros(NumPerElement * len(folders_and_labels), sequence_length, input_size)  # (samples, time_steps, features)
label = torch.zeros(NumPerElement * len(folders_and_labels))

# Counters for the samples processed
count = -1

# Loop through each folder and load the data
for folder_dir, folder_label in folders_and_labels:
    count_in_folder = 0
    for images in os.listdir(folder_dir):
        if images != '.DS_Store' and count_in_folder < NumPerElement:
            # Load the .mat file
            AA = sio.loadmat(os.path.join(folder_dir, images))
            count_in_folder += 1
            count += 1
            for key, value in AA.items():
                if isinstance(value, np.ndarray) and np.iscomplexobj(value):
                    im[count, :, 0] = torch.from_numpy(value.real[0, :])  # Real part
                    im[count, :, 1] = torch.from_numpy(value.imag[0, :].copy())  # Imaginary part
                label[count] = folder_label

        if count_in_folder >= NumPerElement:  # Stop processing after 10 files per folder
            break

# Verify the data shape and labels
print(f"Total samples loaded: {count + 1}")
print(f"Labels for first few samples: {label[:10]}")

# Define the Sequential LSTM Model
model = nn.Sequential(
    nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True),  # Stacked LSTM layers without dropout
    nn.Linear(hidden_size, num_classes)  # Fully connected layer for classification
)

# Define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the device
model = model.to(device)

# Convert to dataset and DataLoader
dataset = TensorDataset(im, label)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 8

# Training loop
for epoch in range(num_epochs):
    model.train()  # Ensure the model is in training mode
    total_loss = 0.0
    total_samples = 0

    all_preds = []
    all_labels = []

    for i, (data, labels) in enumerate(dataloader):
        # Move data and labels to the same device as the model
        data, labels = data.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        lstm_out, _ = model[0](data)  # Only pass through LSTM layer
        outputs = model[1](lstm_out[:, -1, :])  # Use output from last time step (last hidden state)

        # Compute the loss
        loss = criterion(outputs, labels.long())  # Ensure labels are of type long for classification
        total_loss += loss.item() * data.size(0)  # Accumulate loss (weighted by batch size)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        preds = torch.argmax(outputs, dim=1)

        # Collect all predictions and labels for metrics
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

        total_samples += data.size(0)

        if (i+1) % 10 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}")

    # Convert all_preds and all_labels to numpy arrays
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Calculate Precision, Recall, F1 for each class, with zero_division set to 0
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    recall = recall_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    f1 = f1_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)

    # Print the detailed metrics for each class
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {total_loss / total_samples:.4f}, Accuracy: {accuracy:.4f}")
    for i in range(num_classes):
        print(f"Class {i}: Precision = {precision[i]:.4f}, Recall = {recall[i]:.4f}, F1 Score = {f1[i]:.4f}")


Total samples loaded: 35000
Labels for first few samples: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Epoch [1/8], Step [10/1094], Loss: 1.9338
Epoch [1/8], Step [20/1094], Loss: 1.8094
Epoch [1/8], Step [30/1094], Loss: 1.5211
Epoch [1/8], Step [40/1094], Loss: 1.6792
Epoch [1/8], Step [50/1094], Loss: 1.4415
Epoch [1/8], Step [60/1094], Loss: 1.2855
Epoch [1/8], Step [70/1094], Loss: 1.6141
Epoch [1/8], Step [80/1094], Loss: 1.6254
Epoch [1/8], Step [90/1094], Loss: 1.4762
Epoch [1/8], Step [100/1094], Loss: 1.3712
Epoch [1/8], Step [110/1094], Loss: 1.7209
Epoch [1/8], Step [120/1094], Loss: 1.3735
Epoch [1/8], Step [130/1094], Loss: 1.2585
Epoch [1/8], Step [140/1094], Loss: 1.2247
Epoch [1/8], Step [150/1094], Loss: 0.9469
Epoch [1/8], Step [160/1094], Loss: 1.0348
Epoch [1/8], Step [170/1094], Loss: 0.9482
Epoch [1/8], Step [180/1094], Loss: 0.8139
Epoch [1/8], Step [190/1094], Loss: 0.7254
Epoch [1/8], Step [200/1094], Loss: 1.3654
Epoch [1/8], Step [210/1094], Loss: 0.8587

Epoch [2/8], Step [690/1094], Loss: 1.3184
Epoch [2/8], Step [700/1094], Loss: 1.3671
Epoch [2/8], Step [710/1094], Loss: 1.5763
Epoch [2/8], Step [720/1094], Loss: 1.4702
Epoch [2/8], Step [730/1094], Loss: 1.5466
Epoch [2/8], Step [740/1094], Loss: 1.4342
Epoch [2/8], Step [750/1094], Loss: 1.4468
Epoch [2/8], Step [760/1094], Loss: 1.3834
Epoch [2/8], Step [770/1094], Loss: 1.3815
Epoch [2/8], Step [780/1094], Loss: 1.4819
Epoch [2/8], Step [790/1094], Loss: 1.4898
Epoch [2/8], Step [800/1094], Loss: 1.3433
Epoch [2/8], Step [810/1094], Loss: 1.5867
Epoch [2/8], Step [820/1094], Loss: 1.5022
Epoch [2/8], Step [830/1094], Loss: 1.4408
Epoch [2/8], Step [840/1094], Loss: 1.4163
Epoch [2/8], Step [850/1094], Loss: 1.4581
Epoch [2/8], Step [860/1094], Loss: 1.4842
Epoch [2/8], Step [870/1094], Loss: 1.3534
Epoch [2/8], Step [880/1094], Loss: 1.4128
Epoch [2/8], Step [890/1094], Loss: 1.4013
Epoch [2/8], Step [900/1094], Loss: 1.5630
Epoch [2/8], Step [910/1094], Loss: 1.4743
Epoch [2/8]

Epoch [4/8], Step [190/1094], Loss: 1.4560
Epoch [4/8], Step [200/1094], Loss: 1.3575
Epoch [4/8], Step [210/1094], Loss: 1.4189
Epoch [4/8], Step [220/1094], Loss: 1.3026
Epoch [4/8], Step [230/1094], Loss: 1.6936
Epoch [4/8], Step [240/1094], Loss: 1.3448
Epoch [4/8], Step [250/1094], Loss: 1.3993
Epoch [4/8], Step [260/1094], Loss: 1.3714
Epoch [4/8], Step [270/1094], Loss: 1.5089
Epoch [4/8], Step [280/1094], Loss: 1.6169
Epoch [4/8], Step [290/1094], Loss: 1.6234
Epoch [4/8], Step [300/1094], Loss: 1.4722
Epoch [4/8], Step [310/1094], Loss: 1.4726
Epoch [4/8], Step [320/1094], Loss: 1.4009
Epoch [4/8], Step [330/1094], Loss: 1.4979
Epoch [4/8], Step [340/1094], Loss: 1.5111
Epoch [4/8], Step [350/1094], Loss: 1.4325
Epoch [4/8], Step [360/1094], Loss: 1.6767
Epoch [4/8], Step [370/1094], Loss: 1.4846
Epoch [4/8], Step [380/1094], Loss: 1.4347
Epoch [4/8], Step [390/1094], Loss: 1.5120
Epoch [4/8], Step [400/1094], Loss: 1.4101
Epoch [4/8], Step [410/1094], Loss: 1.4074
Epoch [4/8]

Epoch [5/8], Step [890/1094], Loss: 1.2994
Epoch [5/8], Step [900/1094], Loss: 1.3592
Epoch [5/8], Step [910/1094], Loss: 1.5257
Epoch [5/8], Step [920/1094], Loss: 1.4805
Epoch [5/8], Step [930/1094], Loss: 1.6389
Epoch [5/8], Step [940/1094], Loss: 1.5629
Epoch [5/8], Step [950/1094], Loss: 1.5483
Epoch [5/8], Step [960/1094], Loss: 1.4697
Epoch [5/8], Step [970/1094], Loss: 1.4589
Epoch [5/8], Step [980/1094], Loss: 1.6202
Epoch [5/8], Step [990/1094], Loss: 1.4010
Epoch [5/8], Step [1000/1094], Loss: 1.4973
Epoch [5/8], Step [1010/1094], Loss: 1.2816
Epoch [5/8], Step [1020/1094], Loss: 1.4067
Epoch [5/8], Step [1030/1094], Loss: 1.5466
Epoch [5/8], Step [1040/1094], Loss: 1.5844
Epoch [5/8], Step [1050/1094], Loss: 1.4809
Epoch [5/8], Step [1060/1094], Loss: 1.4917
Epoch [5/8], Step [1070/1094], Loss: 1.4998
Epoch [5/8], Step [1080/1094], Loss: 1.5333
Epoch [5/8], Step [1090/1094], Loss: 1.4574
Epoch [5/8], Average Loss: 1.4647, Accuracy: 0.2813
Class 0: Precision = 0.4386, Recall

Epoch [7/8], Step [390/1094], Loss: 1.5353
Epoch [7/8], Step [400/1094], Loss: 1.5396
Epoch [7/8], Step [410/1094], Loss: 1.6389
Epoch [7/8], Step [420/1094], Loss: 1.3563
Epoch [7/8], Step [430/1094], Loss: 1.4516
Epoch [7/8], Step [440/1094], Loss: 1.4062
Epoch [7/8], Step [450/1094], Loss: 1.4152
Epoch [7/8], Step [460/1094], Loss: 1.4586
Epoch [7/8], Step [470/1094], Loss: 1.3433
Epoch [7/8], Step [480/1094], Loss: 1.2959
Epoch [7/8], Step [490/1094], Loss: 1.3616
Epoch [7/8], Step [500/1094], Loss: 1.3820
Epoch [7/8], Step [510/1094], Loss: 1.4886
Epoch [7/8], Step [520/1094], Loss: 1.3685
Epoch [7/8], Step [530/1094], Loss: 1.5648
Epoch [7/8], Step [540/1094], Loss: 1.4550
Epoch [7/8], Step [550/1094], Loss: 1.4551
Epoch [7/8], Step [560/1094], Loss: 1.4625
Epoch [7/8], Step [570/1094], Loss: 1.3453
Epoch [7/8], Step [580/1094], Loss: 1.3708
Epoch [7/8], Step [590/1094], Loss: 1.5102
Epoch [7/8], Step [600/1094], Loss: 1.5439
Epoch [7/8], Step [610/1094], Loss: 1.8074
Epoch [7/8]

Epoch [8/8], Step [1090/1094], Loss: 1.4304
Epoch [8/8], Average Loss: 1.4650, Accuracy: 0.2795
Class 0: Precision = 0.4337, Recall = 0.3902, F1 Score = 0.4108
Class 1: Precision = 0.2104, Recall = 0.3432, F1 Score = 0.2609
Class 2: Precision = 0.4564, Recall = 0.5774, F1 Score = 0.5098
Class 3: Precision = 0.1968, Recall = 0.1816, F1 Score = 0.1889
Class 4: Precision = 0.2077, Recall = 0.3058, F1 Score = 0.2474
Class 5: Precision = 0.1798, Recall = 0.0238, F1 Score = 0.0420
Class 6: Precision = 0.1987, Recall = 0.1346, F1 Score = 0.1605


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import scipy.io as sio
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Constants
NumPerElement = 2000  # Limit the number of samples to 10 per folder (for testing)
sequence_length = 128  # The length of each sequence (number of time steps)
input_size = 40  # The number of features per time step
hidden_size = 128  # Number of hidden units per LSTM layer
num_classes = 4  # Number of output classes
num_layers = 4  # Number of stacked LSTM layers

# Folder paths and corresponding labels
folders_and_labels = [
    ('/Users/anuraagthakur/Desktop/Imag/FMCWD1', 0),
    ('/Users/anuraagthakur/Desktop/Imag/FMCWD2', 1),
    ('/Users/anuraagthakur/Desktop/Imag/FMCWD3', 2),
    ('/Users/anuraagthakur/Desktop/Imag/FMCWD4', 3),
    # Add other folders here if needed
]

# Prepare data tensors
im = torch.zeros(NumPerElement * len(folders_and_labels), input_size, sequence_length)  # (samples, features, time_steps)
label = torch.zeros(NumPerElement * len(folders_and_labels))

# Counters for the samples processed
count = -1

# Loop through each folder and load the data
for folder_dir, folder_label in folders_and_labels:
    count_in_folder = 0
    for images in os.listdir(folder_dir):
        if images != '.DS_Store' and count_in_folder < NumPerElement:
            # Load the .mat file
            AA = sio.loadmat(os.path.join(folder_dir, images))
            count_in_folder += 1
            count += 1
            for key, value in AA.items():
                # Assume value is a (128, 40) array representing 128 time steps and 40 features
                if len(value.shape) == 2 and value.shape[0] == 128 and value.shape[1] == 40:
                    # Transpose the (128, 40) array to (40, 128) for feature and time step flip
                    flipped_data = value.T  # Shape becomes (40, 128)
                    im[count, :, :] = torch.from_numpy(flipped_data)  # Store flipped data
                else:
                    print(f"Unexpected shape for {images}: {value.shape}")
            label[count] = folder_label

        if count_in_folder >= NumPerElement:  # Stop processing after NumPerElement files per folder
            break

# Verify the data shape and labels
print(f"Total samples loaded: {count + 1}")
print(f"Labels for first few samples: {label[:10]}")

# Define the Sequential LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)  # Fully connected layer for classification

    def forward(self, x):
        # Pass through LSTM layer
        lstm_out, _ = self.lstm(x)
        # Use output from last time step (last hidden state)
        out = self.fc(lstm_out[:, -1, :])
        return out

# Initialize the model
model = LSTMModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes)

# Define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Convert to dataset and DataLoader
dataset = TensorDataset(im, label)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    model.train()  # Ensure the model is in training mode
    total_loss = 0.0
    total_samples = 0

    all_preds = []
    all_labels = []

    for i, (data, labels) in enumerate(dataloader):
        # Move data and labels to the same device as the model
        data, labels = data.to(device), labels.to(device)

        # Forward pass
        outputs = model(data)  # Pass data through LSTM model

        # Compute the loss
        loss = criterion(outputs, labels.long())  # Ensure labels are of type long for classification
        total_loss += loss.item() * data.size(0)  # Accumulate loss (weighted by batch size)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        preds = torch.argmax(outputs, dim=1)

        # Collect all predictions and labels for metrics
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

        total_samples += data.size(0)

        if (i + 1) % 10 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(dataloader)}], Loss: {loss.item():.4f}")

    # Convert all_preds and all_labels to numpy arrays
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Calculate Precision, Recall, F1 for each class, with zero_division set to 0
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    recall = recall_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    f1 = f1_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)

    # Print the detailed metrics for each class
    print(f"Epoch [{epoch + 1}/{num_epochs}], Average Loss: {total_loss / total_samples:.4f}, Accuracy: {accuracy:.4f}")
    for i in range(num_classes):
        print(f"Class {i}: Precision = {precision[i]:.4f}, Recall = {recall[i]:.4f}, F1 Score = {f1[i]:.4f}")


Total samples loaded: 8000
Labels for first few samples: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import scipy.io as sio
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Constants
NumPerElement = 2000  # Limit the number of samples to 10 per folder (for testing)
sequence_length = 128  # The length of each sequence (number of time steps)
input_size = 40  # The number of features per time step
hidden_size = 128  # Number of hidden units per LSTM layer
num_classes = 4  # Number of output classes
num_layers = 4  # Number of stacked LSTM layers

# Folder paths and corresponding labels
folders_and_labels = [
    ('/Users/anuraagthakur/Desktop/Imag/FMCWD1', 0),
    ('/Users/anuraagthakur/Desktop/Imag/FMCWD2', 1),
    ('/Users/anuraagthakur/Desktop/Imag/FMCWD3', 2),
    ('/Users/anuraagthakur/Desktop/Imag/FMCWD4', 3),
    # Add other folders here if needed
]

# Prepare data tensors
im = torch.zeros(NumPerElement * len(folders_and_labels), sequence_length, input_size)  # (samples, time_steps, features)
label = torch.zeros(NumPerElement * len(folders_and_labels))

# Counters for the samples processed
count = -1

# Loop through each folder and load the data
for folder_dir, folder_label in folders_and_labels:
    count_in_folder = 0
    for images in os.listdir(folder_dir):
        if images != '.DS_Store' and count_in_folder < NumPerElement:
            # Load the .mat file
            AA = sio.loadmat(os.path.join(folder_dir, images))
            count_in_folder += 1
            count += 1
            for key, value in AA.items():
                # Assume value is a (128, 40) array representing 128 time steps and 40 features
                if len(value.shape) == 2 and value.shape[0] == 128 and value.shape[1] == 40:
                    # Flatten the (128, 40) array into a 1D vector and assign it to the tensor
                    flattened_data = value.flatten()  # Shape becomes (128 * 40,)
                    im[count, :, :] = torch.from_numpy(flattened_data).view(sequence_length, input_size)  # Reshape to (128, 40)
                else:
                    print(f"Unexpected shape for {images}: {value.shape}")
            label[count] = folder_label

        if count_in_folder >= NumPerElement:  # Stop processing after NumPerElement files per folder
            break

# Verify the data shape and labels
print(f"Total samples loaded: {count + 1}")
print(f"Labels for first few samples: {label[:10]}")

# Define the Sequential LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)  # Fully connected layer for classification

    def forward(self, x):
        # Pass through LSTM layer
        lstm_out, _ = self.lstm(x)
        # Use output from last time step (last hidden state)
        out = self.fc(lstm_out[:, -1, :])
        return out

# Initialize the model
model = LSTMModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes)

# Define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Convert to dataset and DataLoader
dataset = TensorDataset(im, label)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    model.train()  # Ensure the model is in training mode
    total_loss = 0.0
    total_samples = 0

    all_preds = []
    all_labels = []

    for i, (data, labels) in enumerate(dataloader):
        # Move data and labels to the same device as the model
        data, labels = data.to(device), labels.to(device)

        # Forward pass
        outputs = model(data)  # Pass data through LSTM model

        # Compute the loss
        loss = criterion(outputs, labels.long())  # Ensure labels are of type long for classification
        total_loss += loss.item() * data.size(0)  # Accumulate loss (weighted by batch size)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        preds = torch.argmax(outputs, dim=1)

        # Collect all predictions and labels for metrics
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

        total_samples += data.size(0)

        if (i + 1) % 10 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(dataloader)}], Loss: {loss.item():.4f}")

    # Convert all_preds and all_labels to numpy arrays
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Calculate Precision, Recall, F1 for each class, with zero_division set to 0
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    recall = recall_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    f1 = f1_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)

    # Print the detailed metrics for each class
    print(f"Epoch [{epoch + 1}/{num_epochs}], Average Loss: {total_loss / total_samples:.4f}, Accuracy: {accuracy:.4f}")
    for i in range(num_classes):
        print(f"Class {i}: Precision = {precision[i]:.4f}, Recall = {recall[i]:.4f}, F1 Score = {f1[i]:.4f}")


Total samples loaded: 8000
Labels for first few samples: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Epoch [1/5], Step [10/250], Loss: 1.3921
Epoch [1/5], Step [20/250], Loss: 1.3893
Epoch [1/5], Step [30/250], Loss: 1.3800
Epoch [1/5], Step [40/250], Loss: 0.9711
Epoch [1/5], Step [50/250], Loss: 1.5563
Epoch [1/5], Step [60/250], Loss: 1.3703
Epoch [1/5], Step [70/250], Loss: 1.3818
Epoch [1/5], Step [80/250], Loss: 1.3870
Epoch [1/5], Step [90/250], Loss: 2.4115
Epoch [1/5], Step [100/250], Loss: 1.3989
Epoch [1/5], Step [110/250], Loss: 1.4384
Epoch [1/5], Step [120/250], Loss: 1.3968
Epoch [1/5], Step [130/250], Loss: 1.3968
Epoch [1/5], Step [140/250], Loss: 1.3546
Epoch [1/5], Step [150/250], Loss: 1.3297
Epoch [1/5], Step [160/250], Loss: 1.1937
Epoch [1/5], Step [170/250], Loss: 0.9313
Epoch [1/5], Step [180/250], Loss: 0.9662
Epoch [1/5], Step [190/250], Loss: 1.3353
Epoch [1/5], Step [200/250], Loss: 1.6262
Epoch [1/5], Step [210/250], Loss: 1.1759
Epoch [1/5], Step [22

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import scipy.io as sio
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Constants
NumPerElement = 2000  # Limit the number of samples to 10 per folder (for testing)
sequence_length = 128  # The length of each sequence (number of time steps)
input_size = 40  # The number of features per time step
hidden_size = 128  # Number of hidden units per LSTM layer
num_classes = 4  # Number of output classes
num_layers = 4  # Number of stacked LSTM layers

# Folder paths and corresponding labels
folders_and_labels = [
    ('/Users/anuraagthakur/Desktop/Imag/2CW', 0),
    ('/Users/anuraagthakur/Desktop/Imag/4CW', 1),
    ('/Users/anuraagthakur/Desktop/Imag/6CW', 2),
    ('/Users/anuraagthakur/Desktop/Imag/8CW', 3),
    # Add other folders here if needed
]

# Prepare data tensors
im = torch.zeros(NumPerElement * len(folders_and_labels), sequence_length, input_size)  # (samples, time_steps, features)
label = torch.zeros(NumPerElement * len(folders_and_labels))

# Counters for the samples processed
count = -1

# Loop through each folder and load the data
for folder_dir, folder_label in folders_and_labels:
    count_in_folder = 0
    for images in os.listdir(folder_dir):
        if images != '.DS_Store' and count_in_folder < NumPerElement:
            # Load the .mat file
            AA = sio.loadmat(os.path.join(folder_dir, images))
            count_in_folder += 1
            count += 1
            for key, value in AA.items():
                # Assume value is a (128, 40) array representing 128 time steps and 40 features
                if len(value.shape) == 2 and value.shape[0] == 128 and value.shape[1] == 40:
                    # Flatten the (128, 40) array into a 1D vector and assign it to the tensor
                    flattened_data = value.flatten()  # Shape becomes (128 * 40,)
                    im[count, :, :] = torch.from_numpy(flattened_data).view(sequence_length, input_size)  # Reshape to (128, 40)
                else:
                    print(f"Unexpected shape for {images}: {value.shape}")
            label[count] = folder_label

        if count_in_folder >= NumPerElement:  # Stop processing after NumPerElement files per folder
            break

# Verify the data shape and labels
print(f"Total samples loaded: {count + 1}")
print(f"Labels for first few samples: {label[:10]}")

# Define the Sequential LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)  # Fully connected layer for classification

    def forward(self, x):
        # Pass through LSTM layer
        lstm_out, _ = self.lstm(x)
        # Use output from last time step (last hidden state)
        out = self.fc(lstm_out[:, -1, :])
        return out

# Initialize the model
model = LSTMModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes)

# Define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Convert to dataset and DataLoader
dataset = TensorDataset(im, label)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    model.train()  # Ensure the model is in training mode
    total_loss = 0.0
    total_samples = 0

    all_preds = []
    all_labels = []

    for i, (data, labels) in enumerate(dataloader):
        # Move data and labels to the same device as the model
        data, labels = data.to(device), labels.to(device)

        # Forward pass
        outputs = model(data)  # Pass data through LSTM model

        # Compute the loss
        loss = criterion(outputs, labels.long())  # Ensure labels are of type long for classification
        total_loss += loss.item() * data.size(0)  # Accumulate loss (weighted by batch size)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        preds = torch.argmax(outputs, dim=1)

        # Collect all predictions and labels for metrics
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

        total_samples += data.size(0)

        if (i + 1) % 10 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(dataloader)}], Loss: {loss.item():.4f}")

    # Convert all_preds and all_labels to numpy arrays
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Calculate Precision, Recall, F1 for each class, with zero_division set to 0
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    recall = recall_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    f1 = f1_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)

    # Print the detailed metrics for each class
    print(f"Epoch [{epoch + 1}/{num_epochs}], Average Loss: {total_loss / total_samples:.4f}, Accuracy: {accuracy:.4f}")
    for i in range(num_classes):
        print(f"Class {i}: Precision = {precision[i]:.4f}, Recall = {recall[i]:.4f}, F1 Score = {f1[i]:.4f}")


Total samples loaded: 8000
Labels for first few samples: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Epoch [1/5], Step [10/250], Loss: 1.3969
Epoch [1/5], Step [20/250], Loss: 1.3652
Epoch [1/5], Step [30/250], Loss: 1.3947
Epoch [1/5], Step [40/250], Loss: 1.3908
Epoch [1/5], Step [50/250], Loss: 1.3899
Epoch [1/5], Step [60/250], Loss: 1.4032
Epoch [1/5], Step [70/250], Loss: 1.3813
Epoch [1/5], Step [80/250], Loss: 1.3946
Epoch [1/5], Step [90/250], Loss: 1.3901
Epoch [1/5], Step [100/250], Loss: 1.3874
Epoch [1/5], Step [110/250], Loss: 1.3905
Epoch [1/5], Step [120/250], Loss: 1.4006
Epoch [1/5], Step [130/250], Loss: 1.3786
Epoch [1/5], Step [140/250], Loss: 1.3810
Epoch [1/5], Step [150/250], Loss: 1.3918
Epoch [1/5], Step [160/250], Loss: 1.3865
Epoch [1/5], Step [170/250], Loss: 1.3855
Epoch [1/5], Step [180/250], Loss: 1.3896
Epoch [1/5], Step [190/250], Loss: 1.3872
Epoch [1/5], Step [200/250], Loss: 1.3950
Epoch [1/5], Step [210/250], Loss: 1.3873
Epoch [1/5], Step [22

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import scipy.io as sio
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Constants
NumPerElement = 2000  # Limit the number of samples to 10 per folder (for testing)
sequence_length = 128  # The length of each sequence (number of time steps)
input_size = 40  # The number of features per time step
hidden_size = 128  # Number of hidden units per LSTM layer
num_classes = 4  # Number of output classes
num_layers = 4  # Number of stacked LSTM layers

# Folder paths and corresponding labels
folders_and_labels = [
    ('/Users/anuraagthakur/Desktop/Imag/FMCWUP1', 0),
    ('/Users/anuraagthakur/Desktop/Imag/FMCWUP2', 1),
    ('/Users/anuraagthakur/Desktop/Imag/FMCWUP3', 2),
    ('/Users/anuraagthakur/Desktop/Imag/FMCWUP4', 3),
    # Add other folders here if needed
]

# Prepare data tensors
im = torch.zeros(NumPerElement * len(folders_and_labels), sequence_length, input_size)  # (samples, time_steps, features)
label = torch.zeros(NumPerElement * len(folders_and_labels))

# Counters for the samples processed
count = -1

# Loop through each folder and load the data
for folder_dir, folder_label in folders_and_labels:
    count_in_folder = 0
    for images in os.listdir(folder_dir):
        if images != '.DS_Store' and count_in_folder < NumPerElement:
            # Load the .mat file
            AA = sio.loadmat(os.path.join(folder_dir, images))
            count_in_folder += 1
            count += 1
            for key, value in AA.items():
                # Assume value is a (128, 40) array representing 128 time steps and 40 features
                if len(value.shape) == 2 and value.shape[0] == 128 and value.shape[1] == 40:
                    # Flatten the (128, 40) array into a 1D vector and assign it to the tensor
                    flattened_data = value.flatten()  # Shape becomes (128 * 40,)
                    im[count, :, :] = torch.from_numpy(flattened_data).view(sequence_length, input_size)  # Reshape to (128, 40)
                else:
                    print(f"Unexpected shape for {images}: {value.shape}")
            label[count] = folder_label

        if count_in_folder >= NumPerElement:  # Stop processing after NumPerElement files per folder
            break

# Verify the data shape and labels
print(f"Total samples loaded: {count + 1}")
print(f"Labels for first few samples: {label[:10]}")

# Define the Sequential LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)  # Fully connected layer for classification

    def forward(self, x):
        # Pass through LSTM layer
        lstm_out, _ = self.lstm(x)
        # Use output from last time step (last hidden state)
        out = self.fc(lstm_out[:, -1, :])
        return out

# Initialize the model
model = LSTMModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes)

# Define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Convert to dataset and DataLoader
dataset = TensorDataset(im, label)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    model.train()  # Ensure the model is in training mode
    total_loss = 0.0
    total_samples = 0

    all_preds = []
    all_labels = []

    for i, (data, labels) in enumerate(dataloader):
        # Move data and labels to the same device as the model
        data, labels = data.to(device), labels.to(device)

        # Forward pass
        outputs = model(data)  # Pass data through LSTM model

        # Compute the loss
        loss = criterion(outputs, labels.long())  # Ensure labels are of type long for classification
        total_loss += loss.item() * data.size(0)  # Accumulate loss (weighted by batch size)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        preds = torch.argmax(outputs, dim=1)

        # Collect all predictions and labels for metrics
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

        total_samples += data.size(0)

        if (i + 1) % 10 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(dataloader)}], Loss: {loss.item():.4f}")

    # Convert all_preds and all_labels to numpy arrays
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Calculate Precision, Recall, F1 for each class, with zero_division set to 0
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    recall = recall_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    f1 = f1_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)

    # Print the detailed metrics for each class
    print(f"Epoch [{epoch + 1}/{num_epochs}], Average Loss: {total_loss / total_samples:.4f}, Accuracy: {accuracy:.4f}")
    for i in range(num_classes):
        print(f"Class {i}: Precision = {precision[i]:.4f}, Recall = {recall[i]:.4f}, F1 Score = {f1[i]:.4f}")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/anuraagthakur/Desktop/Imag/FMCWUP2'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import scipy.io as sio
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Constants
NumPerElement = 2000  # Limit the number of samples to 10 per folder (for testing)
sequence_length = 300  # The length of each sequence (number of time steps)
input_size = 2  # The number of features per time step
hidden_size = 128  # Number of hidden units per LSTM layer
num_classes = 4  # Number of output classes
num_layers = 4  # Number of stacked LSTM layers

# Folder paths and corresponding labels
folders_and_labels = [
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/AWGN',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/ImpulseNoise',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/PinkNoise',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/RayleighNoise',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/4QAM/RicianNoise',0),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/AWGN',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/ImpulseNoise',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/PinkNoise',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/RayleighNoise',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM/RicianNoise',1),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/AWGN',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/ImpulseNoise',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/PinkNoise',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/RayleighNoise',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QAM-Flipped/RicianNoise',2),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/AWGN',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/ImpulseNoise',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/PinkNoise',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/RayleighNoise',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/8QPSK/RicianNoise',3),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/AWGN',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/ImpulseNoise',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/PinkNoise',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/RayleighNoise',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK/RicianNoise',4),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/AWGN',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/ImpulseNoise',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/PinkNoise',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/RayleighNoise',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/BPSK-Flipped/RicianNoise',5),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/AWGN',6),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/ImpulseNoise',6),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/PinkNoise',6),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/RayleighNoise',6),
    ('/Users/anuraagthakur/Desktop/untitled folder 7/QPSK/RicianNoise',6),
    # Add other folders here if needed
]

# Prepare data tensors
im = torch.zeros(NumPerElement * len(folders_and_labels), sequence_length, input_size)  # (samples, time_steps, features)
label = torch.zeros(NumPerElement * len(folders_and_labels))

# Counters for the samples processed
count = -1

# Loop through each folder and load the data
for folder_dir, folder_label in folders_and_labels:
    count_in_folder = 0
    for images in os.listdir(folder_dir):
        if images != '.DS_Store' and count_in_folder < NumPerElement:
            # Load the .mat file
            AA = sio.loadmat(os.path.join(folder_dir, images))
            count_in_folder += 1
            count += 1
            for key, value in AA.items():
                # Assume value is a (128, 40) array representing 128 time steps and 40 features
                if len(value.shape) == 2 and value.shape[0] == 128 and value.shape[1] == 40:
                    # Flatten the (128, 40) array into a 1D vector and assign it to the tensor
                    flattened_data = value.flatten()  # Shape becomes (128 * 40,)
                    im[count, :, :] = torch.from_numpy(flattened_data).view(sequence_length, input_size)  # Reshape to (128, 40)
                else:
                    print(f"Unexpected shape for {images}: {value.shape}")
            label[count] = folder_label

        if count_in_folder >= NumPerElement:  # Stop processing after NumPerElement files per folder
            break

# Verify the data shape and labels
print(f"Total samples loaded: {count + 1}")
print(f"Labels for first few samples: {label[:10]}")

# Define the Sequential LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)  # Fully connected layer for classification

    def forward(self, x):
        # Pass through LSTM layer
        lstm_out, _ = self.lstm(x)
        # Use output from last time step (last hidden state)
        out = self.fc(lstm_out[:, -1, :])
        return out

# Initialize the model
model = LSTMModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes)

# Define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Convert to dataset and DataLoader
dataset = TensorDataset(im, label)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    model.train()  # Ensure the model is in training mode
    total_loss = 0.0
    total_samples = 0

    all_preds = []
    all_labels = []

    for i, (data, labels) in enumerate(dataloader):
        # Move data and labels to the same device as the model
        data, labels = data.to(device), labels.to(device)

        # Forward pass
        outputs = model(data)  # Pass data through LSTM model

        # Compute the loss
        loss = criterion(outputs, labels.long())  # Ensure labels are of type long for classification
        total_loss += loss.item() * data.size(0)  # Accumulate loss (weighted by batch size)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        preds = torch.argmax(outputs, dim=1)

        # Collect all predictions and labels for metrics
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

        total_samples += data.size(0)

        if (i + 1) % 10 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(dataloader)}], Loss: {loss.item():.4f}")

    # Convert all_preds and all_labels to numpy arrays
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Calculate Precision, Recall, F1 for each class, with zero_division set to 0
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    recall = recall_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    f1 = f1_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)

    # Print the detailed metrics for each class
    print(f"Epoch [{epoch + 1}/{num_epochs}], Average Loss: {total_loss / total_samples:.4f}, Accuracy: {accuracy:.4f}")
    for i in range(num_classes):
        print(f"Class {i}: Precision = {precision[i]:.4f}, Recall = {recall[i]:.4f}, F1 Score = {f1[i]:.4f}")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import scipy.io as sio
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Constants
NumPerElement = 2000  # Limit the number of samples to 10 per folder (for testing)
sequence_length = 128  # The length of each sequence (number of time steps)
input_size = 40  # The number of features per time step
hidden_size = 128  # Number of hidden units per LSTM layer
num_classes = 4  # Number of output classes
num_layers = 4  # Number of stacked LSTM layers

# Folder paths and corresponding labels
folders_and_labels = [
    ('/Users/anuraagthakur/Desktop/Imag/2CW', 0),
    ('/Users/anuraagthakur/Desktop/Imag/4CW', 1),
    ('/Users/anuraagthakur/Desktop/Imag/6CW', 2),
    ('/Users/anuraagthakur/Desktop/Imag/8CW', 3),
    # Add other folders here if needed
]

# Prepare data tensors
im = torch.zeros(NumPerElement * len(folders_and_labels), sequence_length, input_size)  # (samples, time_steps, features)
label = torch.zeros(NumPerElement * len(folders_and_labels))

# Counters for the samples processed
count = -1

# Loop through each folder and load the data
for folder_dir, folder_label in folders_and_labels:
    count_in_folder = 0
    for images in os.listdir(folder_dir):
        if images != '.DS_Store' and count_in_folder < NumPerElement:
            # Load the .mat file
            AA = sio.loadmat(os.path.join(folder_dir, images))
            count_in_folder += 1
            count += 1
            for key, value in AA.items():
                # Assume value is a (128, 40) array representing 128 time steps and 40 features
                if len(value.shape) == 2 and value.shape[0] == 128 and value.shape[1] == 40:
                    # Flatten the (128, 40) array into a 1D vector and assign it to the tensor
                    flattened_data = value.flatten()  # Shape becomes (128 * 40,)
                    im[count, :, :] = torch.from_numpy(flattened_data).view(sequence_length, input_size)  # Reshape to (128, 40)
                else:
                    print(f"Unexpected shape for {images}: {value.shape}")
            label[count] = folder_label

        if count_in_folder >= NumPerElement:  # Stop processing after NumPerElement files per folder
            break

# Verify the data shape and labels
print(f"Total samples loaded: {count + 1}")
print(f"Labels for first few samples: {label[:10]}")

# Define the Sequential LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)  # Fully connected layer for classification

    def forward(self, x):
        # Pass through LSTM layer
        lstm_out, _ = self.lstm(x)
        # Use output from last time step (last hidden state)
        out = self.fc(lstm_out[:, -1, :])
        return out

# Initialize the model
model = LSTMModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes)

# Define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Convert to dataset and DataLoader
dataset = TensorDataset(im, label)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    model.train()  # Ensure the model is in training mode
    total_loss = 0.0
    total_samples = 0

    all_preds = []
    all_labels = []

    for i, (data, labels) in enumerate(dataloader):
        # Move data and labels to the same device as the model
        data, labels = data.to(device), labels.to(device)

        # Forward pass
        outputs = model(data)  # Pass data through LSTM model

        # Compute the loss
        loss = criterion(outputs, labels.long())  # Ensure labels are of type long for classification
        total_loss += loss.item() * data.size(0)  # Accumulate loss (weighted by batch size)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        preds = torch.argmax(outputs, dim=1)

        # Collect all predictions and labels for metrics
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

        total_samples += data.size(0)

        if (i + 1) % 10 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(dataloader)}], Loss: {loss.item():.4f}")

    # Convert all_preds and all_labels to numpy arrays
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Calculate Precision, Recall, F1 for each class, with zero_division set to 0
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    recall = recall_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)
    f1 = f1_score(all_labels, all_preds, average=None, labels=np.arange(num_classes), zero_division=0)

    # Print the detailed metrics for each class
    print(f"Epoch [{epoch + 1}/{num_epochs}], Average Loss: {total_loss / total_samples:.4f}, Accuracy: {accuracy:.4f}")
    for i in range(num_classes):
        print(f"Class {i}: Precision = {precision[i]:.4f}, Recall = {recall[i]:.4f}, F1 Score = {f1[i]:.4f}")
